In [5]:
from langchain_groq import ChatGroq

In [3]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama3-70b-8192"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

That's an easy one!

The first person to set foot on the Moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the Moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to walk on the Moon.


In [12]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/junior-software-engineer/job/R-38916")
page_data = loader.load().pop().page_content
print(page_data)






















Junior Software Engineer










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



Select a Lang

In [13]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

Here is the extracted job posting in JSON format:

```
[
    {
        "role": "Junior Software Engineer",
        "experience": "Not specified",
        "skills": [
            "Python programming",
            "Version control systems like Git",
            "SQL and Spark basics",
            "Data science concepts and common machine learning algorithms",
            "Cloud computing concepts (e.g., AWS, Google Cloud)",
            "Mathematics, statistics, or computer science"
        ],
        "description": "Developing sophisticated analytics systems that directly impact our business. Working on a cross-disciplinary team (Machine Learning / Software Engineering / DevOps) to enable data-driven decision-making across multiple organizations."
    }
]
```


In [14]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Junior Software Engineer',
  'experience': 'Not specified',
  'skills': ['Python programming',
   'Version control systems like Git',
   'SQL and Spark basics',
   'Data science concepts and common machine learning algorithms',
   'Cloud computing concepts (e.g., AWS, Google Cloud)',
   'Mathematics, statistics, or computer science'],
  'description': 'Developing sophisticated analytics systems that directly impact our business. Working on a cross-disciplinary team (Machine Learning / Software Engineering / DevOps) to enable data-driven decision-making across multiple organizations.'}]

In [16]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstacks,Links
0,"Python, Scikit-learn, TensorFlow, XGBoost",https://github.com/HatakeKakashiD/Insuarance_p...
1,"Python, Elasticsearch, PyTorch, Keras",https://github.com/HatakeKakashiD/Sementic-Sea...
2,"MERN, React.js, Angular",https://github.com/Senash0813/HealthCare-App.git
3,"Android Studio, Kotlin, Flutter",https://github.com/Senash0813/TicTacToe.git
4,"Android Studio, Kotlin, React Native",https://github.com/Senash0813/MAD-Lab04.git
5,"BeautifulSoup, Scrapy, Selenium",https://github.com/Senash0813/web-scraper-reac...


In [17]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstacks"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [20]:
links = collection.query(query_texts=['Experience in Python and React'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://github.com/HatakeKakashiD/Insuarance_predictor_fdm.git'},
  {'links': 'https://github.com/Senash0813/MAD-Lab04.git'}]]

In [23]:
job = json_res

In [24]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Senash Hettiarachchi a 3rd year Data Science undergraduate at SLIIT. You have a strong academic record, 
        earning a place on the Dean’s List for two semesters. You excel in teamwork and have demonstrated exceptional leadership skills.
        Additionally, you have hands-on experience with machine learning projects and a solid understanding of data science concepts.
        Your job is to write a email of interest regarding the job mentioned above describing your capabilities in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase your portfolio: {link_list}
        Remember you are Senash Hettiarachchi, Data Science undergraduate at SLIIT. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Application for Junior Software Engineer Position

Dear Hiring Manager,

I am excited to express my interest in the Junior Software Engineer role at your esteemed organization. As a 3rd-year Data Science undergraduate at SLIIT, I am confident that my academic background, hands-on experience, and leadership skills make me an ideal candidate for this position.

Throughout my academic journey, I have developed a strong foundation in data science concepts, machine learning algorithms, and programming languages such as Python. I have also worked on various projects that demonstrate my ability to work with version control systems like Git and have a solid understanding of SQL and Spark basics. Additionally, I have a good grasp of cloud computing concepts, particularly AWS and Google Cloud.

As a team player with exceptional leadership skills, I have successfully collaborated with peers on multiple projects, showcasing my ability to work effectively in a cross-disciplinary team. My e